# Comparing interpolation speeds between valid kernels

## Usual setup

In [1]:
from helper_loader import *

In [2]:
volume_path = Path("resources/d2783eb27223868f57e159f01ce7a1b2.npz")

In [3]:
atlas_array = load_volume(
    get_structure_mask_path("root", Resolution.MICRONS_25), return_raw_array=True
)
atlas_volume = vedo.Volume(atlas_array)
atlas_volume.cmap(**get_cmap(atlas_volume))

aligned_array = np.load(volume_path)["array"]
aligned_volume = vedo.Volume(aligned_array)
aligned_volume.cmap(**get_cmap(aligned_volume))

masked_array = np.where(atlas_array, aligned_array, 0)
masked_volume = vedo.Volume(masked_array)
masked_volume.cmap(**get_cmap(masked_volume));

In [4]:
focused_area = (
    (aligned_array.shape[0] // 3, aligned_array.shape[0] // 3 * 2),
    (aligned_array.shape[1] // 3, aligned_array.shape[1] // 3 * 2),
    (60, aligned_array.shape[2] // 3),
)
focused_shape = (tmp := np.array(focused_area).T)[1] - tmp[0]
focused_slice = tuple(slice(focus[0], focus[1]) for focus in focused_area)

focused_array = np.zeros_like(atlas_array)
tmp = np.ones(shape=focused_shape, dtype=np.uint16)
draw_array_skeleton(tmp)
focused_array[focused_slice] = tmp

focused_volume = vedo.Volume(focused_array)
focused_volume.cmap(**get_cmap(focused_volume, alpha=[0, 0.5]));

In [5]:
atlas_volume_cropped = vedo.Volume(np.where(focused_array, atlas_array, 0))
atlas_volume_cropped.cmap(**get_cmap(atlas_volume_cropped))

aligned_volume_cropped = vedo.Volume(np.where(focused_array, aligned_array, 0))
aligned_volume_cropped.cmap(**get_cmap(aligned_volume_cropped))

masked_volume_cropped = vedo.Volume(np.where(focused_array, masked_array, 0))
masked_volume_cropped.cmap(**get_cmap(masked_volume_cropped));

In [6]:
known_coordinates = np.nonzero(masked_array)
known_points = np.array(known_coordinates).T

known_values = masked_array[known_coordinates]

target_coordinates = np.nonzero(atlas_array)
target_points = np.array(target_coordinates).T

## Comparisons

In [63]:
neighbours = 5**3
epsilon = 1
point_count = 100_000
iteration_count = 5

times = {}

for kernel in [
    "linear",
    "gaussian",
    "multiquadric",
    "inverse_multiquadric",
    "inverse_quadratic",
]:
    logging.info(f"Computing interpolation for {kernel} kernel.")
    for _ in range(iteration_count):
        interpolator = RBFInterpolator(
            known_points,
            known_values,
            kernel=kernel,
            neighbors=neighbours,
            epsilon=epsilon,
        )
        start_time = perf_counter()
        interpolated_values = interpolator(target_points[:point_count])

        timing = perf_counter() - start_time
        if (timings := times.get(kernel)) is not None:
            timings.append(timing)
        else:
            times[kernel] = [timing]

[2024-09-20 16:11:43] - [    INFO ] - Computing interpolation for linear kernel. (root:15)
[2024-09-20 16:13:55] - [    INFO ] - Computing interpolation for gaussian kernel. (root:15)
[2024-09-20 16:16:51] - [    INFO ] - Computing interpolation for multiquadric kernel. (root:15)
[2024-09-20 16:19:33] - [    INFO ] - Computing interpolation for inverse_multiquadric kernel. (root:15)
[2024-09-20 16:22:39] - [    INFO ] - Computing interpolation for inverse_quadratic kernel. (root:15)


In [66]:
indentation = " " * 4

print(f"Parameters:")
print(f"{indentation}Neighbours = {neighbours}")
print(f"{indentation}Epsilon = {epsilon}")
print(f"{indentation}Point count = {point_count:_}")
print(f"{indentation}Iteration count = {iteration_count}")

print()

mean_times = np.mean(np.array(list(times.values())), axis=1)
std_times = np.std(np.array(list(times.values())), axis=1)
print(f"Results:")
print(f"{indentation}{'Kernel':^20}   {'Time (+/- std)':^18}")
for index, kernel in enumerate(times.keys()):
    print(
        f"{indentation}{kernel.replace('_', ' ').capitalize():<20}: "
        f"{mean_times[index]:>6.2f}s "
        f"(+/- {std_times[index]:>5.2f})"
    )

Parameters:
    Neighbours = 125
    Epsilon = 1
    Point count = 100_000
    Iteration count = 5

Results:
           Kernel            Time (+/- std)  
    Linear              :  25.86s (+/-  1.41)
    Gaussian            :  34.62s (+/-  3.56)
    Multiquadric        :  31.82s (+/-  5.61)
    Inverse multiquadric:  36.69s (+/-  5.98)
    Inverse quadratic   :  29.38s (+/-  2.04)
